- sql 데이터를 pandas DataFrame으로 읽어오기

In [1]:
import pymysql
import pandas as pd

In [2]:
# db 연결
db = pymysql.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    passwd = '1234',
    db = 'student_mgmt',
    charset = 'utf8'
)

pd.read_sql(쿼리, 연결 된 db connetion 객체)

In [3]:
sql = "SHOW TABLES"
df = pd.read_sql(sql, db)

C:\Users\yju12\AppData\Local\Temp\ipykernel_6712\1302501115.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


In [5]:
df

,Tables_in_student_mgmt
0,students


In [6]:
# student_mgmt 스키마의 students 테이블의 모든 컬럼 불러오기

sql = "SELECT * FROM students"
df = pd.read_sql(sql, db)
df

C:\Users\yju12\AppData\Local\Temp\ipykernel_6712\1391524238.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, db)


,id,name,gender,birth,english,math,korean
0,1,dave,man,1983-07-16,90,80,71
1,2,minsun,woman,1982-10-16,30,88,60
2,3,david,man,1982-12-10,78,77,30
3,4,jade,man,1979-11-01,45,66,20
4,5,jane,man,1990-11-12,65,32,90
5,6,wage,woman,1982-01-13,76,30,80
6,7,tina,woman,1982-12-03,87,62,71


In [7]:
# 불러 온 df를 csv로 저장 후 db접속 종료.
df.to_csv('students.csv', index = False, encoding = 'utf-8')
db.close()

---

- 외래키와 기본키 : 데이터 무결성 실습

In [8]:
# db 접속 (sqlDB)
db = pymysql.connect(
    host = 'localhost',
    port = 3306,
    user = 'root',
    passwd = '1234',
    db = 'sqlDB',
    charset = 'utf8'
)

In [10]:
# sqlDB의 userTbl 테이블을 DataFrame 형태로 불러오기  
sql = "SELECT * FROM userTbl"
df_user = pd.read_sql(sql, db)
df_user

C:\Users\yju12\AppData\Local\Temp\ipykernel_6712\2269796934.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_user = pd.read_sql(sql, db)


,userID,name,birthYear,addr,mobile1,mobile2,height,mDate
0,BBK,바비킴,1973,서울,010,0000000,176,2013-05-05
1,EJW,은지원,1972,경북,011,8888888,174,2014-03-03
2,JKW,조관우,1965,경기,018,9999999,172,2010-10-10
3,JYP,조용필,1950,경기,011,4444444,166,2009-04-04
4,KBS,김범수,1979,경남,011,2222222,173,2012-04-04
5,KKH,김경호,1971,전남,019,3333333,177,2007-07-07
6,LJB,임재범,1963,서울,016,6666666,182,2009-09-09
7,LSG,이승기,1987,서울,011,1111111,182,2008-08-08
8,SSK,성시경,1979,서울,None,None,186,2013-12-12
9,YJS,윤종신,1969,경남,None,None,170,2005-05-05


In [11]:
# sqlDB의 userTbl 테이블을 DataFrame 형태로 불러오기  
sql = "SELECT * FROM buyTbl"
df_buy = pd.read_sql(sql, db)
df_buy

C:\Users\yju12\AppData\Local\Temp\ipykernel_6712\3378051000.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_buy = pd.read_sql(sql, db)


,num,userID,prodName,groupName,price,amount
0,1,KBS,운동화,None,30,2
1,2,KBS,노트북,전자,1000,1
2,3,JYP,모니터,전자,200,1
3,4,BBK,모니터,전자,200,5
4,5,KBS,청바지,의류,50,3
5,6,BBK,메모리,전자,80,10
6,7,SSK,책,서적,15,5
7,8,EJW,책,서적,15,2
8,9,EJW,청바지,의류,50,1
9,10,BBK,운동화,None,30,2


userTbl의 'userID'가 PK<br>
buyTbl의 'userID'는 userTbl을 참조하는 FK
- userTbl에 없는 'userID'를 buyTbl에 추가해보기

In [13]:
# cursor = db.cursor()
# sql = """
#     INSERT INTO buyTbl(userID, prodName, groupName, price, amount)
#     VALUES('STJ', '운동화', NULL, 30, 2)
# """
# cursor.execute(sql)
# db.commit()


# 결과 : Cannot add or update a child row: a foreign key constraint fails 오류 발생
# -> fk의 데이터 무결성으로 인해 오류가 발생

userTbl의 'userID'에 'STJ'가 있어야 함.
- 'userID'에 'STJ' 추가 후 buyTbl에 'STJ'데이터 추가해보기

In [15]:
cursor = db.cursor()
sql = """
    INSERT INTO userTbl VALUES('STJ', '서태지', 1975, '경기', '011', '00000000', '171', '2013-5-5')
"""
cursor.execute(sql)
db.commit()

In [17]:
sql = """
    INSERT INTO buyTbl(userID, prodName, groupName, price, amount)
    VALUES('STJ', '운동화', NULL, 30, 2)
"""
cursor.execute(sql)
db.commit()

## 성공적으로 실행됨을 확인

- userTbl에 있는 'STJ'데이터 삭제

In [20]:
# sql = """
#     DELETE FROM userTbl where userID = 'STJ'
# """
# cursor.execute(sql)
# db.commit()

## 결과 : Cannot delete or update a parent row: a foreign key constraint fails 오류 발생
## buyTbl에 해당 'STJ'를 참조하는 데이터가 있으므로 삭제 불가.

- userTbl에 있는 'userID'를 buyTbl에 추가

In [14]:
cursor = db.cursor()
sql = """
    INSERT INTO buyTbl(userID, prodName, groupName, price, amount)
    VALUES('BBK', '운동화', NULL, 30, 2)
"""
cursor.execute(sql)
db.commit()

# 결과 : 성공적으로 실행됨.